# Zoningtaxlots QAQC

### Objectives:
+ Combine `qc_versioncomparison.csv` with `qc_versioncomparisonnownullcount.csv`; sort by field name. (Resulting report will show __if the value changed to a different value or to/from a null value__).
+ Add a __difference__ column to `qc_frequencychanges.csv`; sort by field name.
+ Add two fields to the BBL diff report
    + Flag indicating that __lot intersects with a rezoning done since the last version__
    + Flag indicating that __the area of the lot (taken from DTM) has changed by more than +/- 10% since the last version__
+ Rename fields in BBL diff report for the fields showing the new data, using similar naming convention as used for previous data set, e.g., ZD1NEW.

In [29]:
import geopandas as gpd
import pandas as pd
import os
from sqlalchemy import create_engine
from pathlib import Path
import time
from shapely.wkb import dumps, loads
from shapely.wkt import loads as wkt_loads 

pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 50)
print(time.strftime("%m/%d/%Y %H:%M:%S"))

11/04/2019 20:19:09


In [30]:
%load_ext dotenv
%dotenv ../.env

The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


In [31]:
conn = create_engine(os.getenv('BUILD_ENGINE').replace('localhost', os.getenv('IP')))

In [32]:
recipe_conn = create_engine(os.getenv('RECIPE_ENGINE'))

In [33]:
# Reports the number of records that experienced a change in the value
query = '''
select field, count as diff_count, 
percent as diff_percent,  
        newnullcount, oldnullcount, 
        countnew as total_count_new, 
        countold as total_count_old, 
        (countnew-countold) as total_count_diff 
from(
    SELECT * FROM
        ztl_qc_versioncomparisoncount a 
    JOIN 
        ztl_qc_versioncomparisonnownullcount b
    USING (field)) c
JOIN
frequencychanges d
USING (field)
ORDER BY field;
'''
df_versioncomparison = pd.read_sql(sql=query, con=conn)
df_versioncomparison

,field,diff_count,diff_percent,newnullcount,oldnullcount,total_count_new,total_count_old,total_count_diff
0,commercialoverlay1,27.0,0.00,3,3,74971,74961,10
1,commercialoverlay2,0.0,0.00,0,0,165,165,0
2,limitedheightdistrict,0.0,0.00,0,0,3037,3037,0
3,specialdistrict1,3.0,0.00,0,0,101895,101896,-1
4,specialdistrict2,0.0,0.00,0,0,80,81,-1
5,specialdistrict3,0.0,0.00,0,0,0,0,0
6,zoningdistrict1,418.0,0.02,0,0,858349,858362,-13
7,zoningdistrict2,18.0,0.00,7,7,19868,19865,3
8,zoningdistrict3,0.0,0.00,0,0,206,206,0
9,zoningdistrict4,0.0,0.00,0,0,13,13,0


In [34]:
# Reports the full zoning comarison table
query = '''
SELECT bblnew, bblprev, 
        zd1new, zd1prev, zd2new, zd2prev, zd3new, zd3prev, zd4new, zd4prev, 
        zmcnew, zmcprev, zmnnew, zmnprev, 
        co1new, co1prev, co2new, co2prev, 
        sd1new, sd1prev, sd2new, sd2prev, sd3new, sd3prev, 
        lhdnew, lhdprev, 
        inzonechange, mihflag, mihoption, 
        geom from bbldiffs;
'''
bbldiffs = gpd.GeoDataFrame.from_postgis(sql=query, con=conn)
bbldiffs

,bblnew,bblprev,zd1new,zd1prev,zd2new,zd2prev,zd3new,zd3prev,zd4new,zd4prev,zmcnew,zmcprev,zmnnew,zmnprev,co1new,co1prev,co2new,co2prev,sd1new,sd1prev,sd2new,sd2prev,sd3new,sd3prev,lhdnew,lhdprev,inzonechange,mihflag,mihoption,geom
0,4066240019,4066240019,R2X,R2,None,None,None,None,None,None,None,None,14C,14C,None,None,None,None,None,None,None,None,None,None,None,None,Y,None,None,"MULTIPOLYGON (((-73.82158 40.72430, -73.82144 ..."
1,4066010044,4066010044,R2X,R2,None,None,None,None,None,None,None,None,14A,14A,None,None,None,None,None,None,None,None,None,None,None,None,Y,None,None,"MULTIPOLYGON (((-73.82401 40.72429, -73.82367 ..."
2,4066040073,4066040073,R2X,R2,None,None,None,None,None,None,None,None,14C,14C,None,None,None,None,None,None,None,None,None,None,None,None,Y,None,None,"MULTIPOLYGON (((-73.82314 40.72289, -73.82303 ..."
3,4066020024,4066020024,R2X,R2,None,None,None,None,None,None,None,None,14A,14A,None,None,None,None,None,None,None,None,None,None,None,None,Y,None,None,"MULTIPOLYGON (((-73.82425 40.72430, -73.82420 ..."
4,4066110014,4066110014,R2X,R2,None,None,None,None,None,None,None,None,14C,14C,None,None,None,None,None,None,None,None,None,None,None,None,Y,None,None,"MULTIPOLYGON (((-73.82269 40.71811, -73.82269 ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
447,4006450046,4006450046,R6A,M1-1,None,None,None,None,None,None,None,None,9B,9B,C1-3,None,None,None,None,None,None,None,None,None,None,None,Y,None,None,"MULTIPOLYGON (((-73.92268 40.75646, -73.92237 ..."
448,4066040014,4066040014,R2X,R2,None,None,None,None,None,None,None,None,14A,14A,None,None,None,None,None,None,None,None,None,None,None,None,Y,None,None,"MULTIPOLYGON (((-73.82375 40.72361, -73.82386 ..."
449,1004640058,1004640058,R8B,R8B,C6-1,None,None,None,None,None,None,None,12C,12C,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,"MULTIPOLYGON (((-73.98932 40.72938, -73.98940 ..."
450,4066230061,4066230061,R4,R4,R2X,None,None,None,None,None,None,None,14C,14C,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,"MULTIPOLYGON (((-73.82269 40.72451, -73.82276 ..."


## DTM Comparison

In [35]:
version_old = '2019/10/02'
version_new = '2019/11/04'

In [36]:
# Reports lots that had an area change
query = f'''
with dtm_compare as (
    SELECT bbl, geom_new, geom_old, (case when geom_new = geom_old then 0 else 1 end) flag 
    FROM 
    (SELECT bbl, ST_Multi(ST_Union(f.wkb_geometry)) as geom_new 
        FROM dof_dtm."{version_new}" f GROUP BY bbl ) a
    JOIN 
    (SELECT bbl, ST_Multi(ST_Union(f.wkb_geometry)) as geom_old 
        FROM dof_dtm."{version_old}" f GROUP BY bbl ) b
    USING(bbl))
, changed as (
    SELECT *, (st_area(geom_new)-st_area(geom_old))/st_area(geom_old) as area_diff 
    FROM dtm_compare
    WHERE flag = 1)
SELECT * FROM changed WHERE area_diff > 0.1 OR area_diff < -0.1;
'''

In [37]:
bbl_areachange = gpd.GeoDataFrame.from_postgis(sql=query, con=recipe_conn, geom_col='geom_new')

In [38]:
bbl_areachange_new = bbl_areachange[['bbl', 'geom_new', 'area_diff']]

In [39]:
bbl_areachange_old = bbl_areachange[['bbl', 'geom_old', 'area_diff']]
bbl_areachange_old.loc[:, 'geom_old'] = bbl_areachange_old['geom_old'].apply(lambda x: wkt_loads(loads(x,  hex=True).wkt))
bbl_areachange_old=gpd.GeoDataFrame(bbl_areachange_old, geometry='geom_old')

/opt/conda/lib/python3.7/site-packages/pandas/core/indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [45]:
from ipyleaflet import Map, basemaps, GeoData, basemap_to_tiles, LayersControl, FullScreenControl, Popup, Marker
from ipywidgets import HTML

m = Map(center=(40.730610, -73.935242), zoom=11)

dark_matter_layer = basemap_to_tiles(basemaps.CartoDB.DarkMatter, close_popup_on_click=True)
m.add_layer(dark_matter_layer)

In [46]:
new = GeoData(geo_dataframe = bbl_areachange_new, 
              style={'color': 'green', 'opacity':10, 'weight':1.9, 'dashArray':'2', 'fillOpacity':0.3},
              name = 'new')

old = GeoData(geo_dataframe = bbl_areachange_old, 
              style={'color': 'red', 'opacity':10, 'weight':1.9, 'dashArray':'2', 'fillOpacity':0.3}, 
              name = 'old')

In [47]:
m.add_layer(new)
m.add_layer(old)
m.add_control(LayersControl())
m.add_control(FullScreenControl())

In [48]:
for i in range(bbl_areachange_new.shape[0]):
    center = (bbl_areachange_new.loc[i, 'geom_new'].centroid.y,
              bbl_areachange_new.loc[i, 'geom_new'].centroid.x)
    bbl = str(list(bbl_areachange_new['bbl'])[i])
    area_change = round(list(bbl_areachange_new['area_diff'])[i]*100, 2)
    marker = Marker(location=center)
    m.add_layer(marker)
    marker.popup = HTML(value=f'''<a href=https://zola.planning.nyc.gov/bbl/{bbl}> {bbl} </a>
                                  <p> area change: {area_change}% </p>''')

In [49]:
m

Map(basemap={'url': 'https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png', 'max_zoom': 19, 'attribution': 'Map …